In [2]:
# Import all the dependencies
from qdrant_client import models, QdrantClient
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader
from langchain.vectorstores.qdrant import Qdrant
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain import PromptTemplate
from peft import PeftModel, PeftConfig
#
from tqdm.auto import tqdm
from uuid import uuid4
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from sentence_transformers import SentenceTransformer, util
import pandas as pd

#
import os
import random
import torch

In [17]:
# Load the document that you need to parse, please change the location to where the pdf resides

# Load 1 PDF file


# loader = PyPDFDirectoryLoader("/mnt/data/" + os.environ['DOMINO_PROJECT_NAME'] + "/RAG/" + pdf_name)

# or load an entire folder
loader = PyPDFDirectoryLoader("/mnt/data/" + os.environ['DOMINO_PROJECT_NAME'] + "/RAG_Docs/")
data = loader.load_and_split(RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0))

In [18]:
print(f"There are {len(data)} pages in the document")

There are 60 pages in the document


In [19]:
# Pick a sample page
print(data[random.randint(0, len(data))])

page_content='segmentation bias. Model weights for each of the diﬀerent pre-segmentation models will be made publicly\navailable at the conclusion of the challenge.\n2.7 Manual corrections\nFor each meningioma case, manual review and correction of pre-segmented labels was performed by\nindividual volunteer “annotators” with widely varying experience levels spanning from medical students\nto fellowship trained neuroradiologists with 10+ years of experience. Manual corrections were performed\nusing ITK-SNAP, a free, open-source, multi-platform software application used to segment structures\nin 3D and 4D biomedical images [19]. Annotators were provided with each of the following: 1) basic\ninstruction on using ITK-SNAP for meningioma segmentation, 2) written descriptions of the composition\nof each tumor sub-compartment, and 3) a list (with examples) of common pre-segmentation errors to\nidentify and address. These steps were designed to reduce inter-observer variability in tumor segment

In [20]:
# Split the data into pages
metadatas = []
texts = []
for row in data:
  metadatas.append(row.metadata)
  texts.append(row.page_content)
print(len(metadatas),len(texts))

60 60


In [21]:
# Setup the prompt template to use for the QA bot
prompt_template = """Use the following pieces of context to answer the question enclosed within  3 backticks at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
Please provide an answer which is factually correct and based on the information retrieved from the vector store.
Please also mention any quotes supporting the answer if any present in the context supplied within two double quotes "" .

{context}

QUESTION:```{question}```
ANSWER:
"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context","question"])
#


In [22]:
# Load the embedding model
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
embedding_model_name = "BAAI/bge-small-en"
os.environ['SENTENCE_TRANSFORMERS_HOME'] = '/mnt/data/' + os.environ['DOMINO_PROJECT_NAME'] + '/model_cache/'
embeddings = HuggingFaceBgeEmbeddings(model_name="BAAI/bge-small-en",
                                      model_kwargs=model_kwargs,
                                      encode_kwargs=encode_kwargs
                                     )

.gitattributes: 100%|██████████| 1.52k/1.52k [00:00<00:00, 216kB/s]
1_Pooling/config.json: 100%|██████████| 190/190 [00:00<00:00, 27.6kB/s]
README.md: 100%|██████████| 90.2k/90.2k [00:00<00:00, 1.32MB/s]
config_sentence_transformers.json: 100%|██████████| 124/124 [00:00<00:00, 56.7kB/s]
model.safetensors: 100%|██████████| 133M/133M [00:00<00:00, 206MB/s]  
pytorch_model.bin: 100%|██████████| 134M/134M [00:00<00:00, 274MB/s] 
sentence_bert_config.json: 100%|██████████| 52.0/52.0 [00:00<00:00, 6.78kB/s]
special_tokens_map.json: 100%|██████████| 125/125 [00:00<00:00, 58.6kB/s]
tokenizer.json: 100%|██████████| 711k/711k [00:00<00:00, 50.9MB/s]
tokenizer_config.json: 100%|██████████| 366/366 [00:00<00:00, 171kB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 53.1MB/s]
modules.json: 100%|██████████| 349/349 [00:00<00:00, 162kB/s]


In [23]:
# Uncomment this code if you want to store the embeddings in Qdrant in-memory
doc_store = Qdrant.from_texts(texts,
                              metadatas=metadatas,
                              embedding=embeddings,
                              location=":memory:",
                              collection=f"{embedding_model_name}_arxiv_paper")

In [24]:
 # Persist the embeddings to disk
doc_store = Qdrant.from_texts(texts,
                              metadatas=metadatas,
                              embedding=embeddings,
                              path="/mnt/artifacts/local_qdrant/",
                              prefer_grpc=True,
                              collection=f"{embedding_model_name}_arxiv_paper")

In [25]:
# Load the model and the tokenizer
chain_type_kwargs = {"prompt": PROMPT}


################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

model_id = "NousResearch/Llama-2-7b-chat-hf"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    cache_dir="/mnt/data/" + os.environ['DOMINO_PROJECT_NAME'] + "/model_cache/",
    quantization_config=bnb_config,
    device_map='auto'
)

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
tokenizer.pad_token_id = model.config.eos_token_id

config.json: 100%|██████████| 583/583 [00:00<00:00, 269kB/s]
model.safetensors.index.json: 100%|██████████| 26.8k/26.8k [00:00<00:00, 10.8MB/s]
model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]
model-00001-of-00002.safetensors:   0%|          | 10.5M/9.98G [00:00<03:45, 44.2MB/s]
model-00001-of-00002.safetensors:   0%|          | 41.9M/9.98G [00:00<01:10, 142MB/s] 
model-00001-of-00002.safetensors:   1%|          | 73.4M/9.98G [00:00<00:50, 195MB/s]
model-00001-of-00002.safetensors:   1%|          | 105M/9.98G [00:00<00:49, 199MB/s] 
model-00001-of-00002.safetensors:   1%|▏         | 136M/9.98G [00:00<00:47, 208MB/s]
model-00001-of-00002.safetensors:   2%|▏         | 168M/9.98G [00:00<00:42, 230MB/s]
model-00001-of-00002.safetensors:   2%|▏         | 199M/9.98G [00:00<00:39, 247MB/s]
model-00001-of-00002.safetensors:   2%|▏         | 231M/9.98G [00:01<00:40, 241MB/s]
model-00001-of-00002.safetensors:   3%|▎         | 262M/9.98G [00:01<00:38, 254MB/s]
model-0

In [26]:
# Setup the QA chain
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=100)
rag_llm = HuggingFacePipeline(pipeline=pipe)
    
qa_chain = RetrievalQA.from_chain_type(llm=rag_llm,
                                       chain_type="stuff",
                                       chain_type_kwargs={"prompt": PROMPT},
                                       retriever=doc_store.as_retriever(search_kwargs={"k": 5}),
                                       return_source_documents=True
                                      )

In [27]:
# Ask a question
user_question = input("Please provide your question here :")
result = qa_chain(user_question)
result['result']

# What is a Meningioma?

Please provide your question here : What is a Meningioma?


/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


'A meningioma is a type of brain tumor that originates in the meninges, the protective membranes that cover the brain and spinal cord. Meningiomas are the most common type of primary brain tumor, accounting for approximately 30% of all brain tumors. They can occur in different locations in the brain, including the convexity, falcine, parasagittal, intraventricular, and skull base'